# Analysis - Object Detection

This notebook is about the exploratory data analysis to Object Detection project

In [ ]:
import os
import random

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

import json

import project.download_content as content

from collections import Counter

import notebooks_utils.analysis as utils

from IPython.display import display




%matplotlib inline

This analysis starts with the downloading of all the data. To do this, you could use the makefile created. Just open the terminal, go to where you clone this project and run `make download-content`, following the instructions to download files.

**To run this analysis and reproduced it, you must download the METADATA files**. Besides that, in some cells, it is necessary to download the images (TRAIN, TEST, and VALIDATION image files). Because of that, to reproduce this analysis entirely, you should download these files also (download around 550Gb). If you did not download them, these cells are not going to run entirely, but the cell will notify you about this, and the process is going to follow.

The analysis made with all the images files downloaded could be accessed in an HTML file that is in the project, called "analysis.html". You do not need to download all images to see it, open the file in your browser.

## Wrangling

### Gather

In [ ]:
# Gathering all files
if not content.does_metadata_exist():
    raise OSError(f'There are metadata file(s) that did not downloaded yet...')
print('All Metadata files exist...')

METAPATH = os.path.join(content.DATAPATH, 'METADATA')

# metadata general files
print('Gathering all metadata files...', end='')

df_classes_raw = pd.read_csv(METAPATH + "/class-descriptions-boxable.csv",
                             names=['class_encode', 'class_name'],
                             index_col='class_encode')
with open(METAPATH + "/bbox_labels_600_hierarchy.json") as f:
    dict_hierarchy_raw = json.load(f)
print('OK!')


# train files
print('Gathering all train files...', end='')
df_train_bbox_raw = pd.read_csv(METAPATH + "/train-annotations-bbox.csv")
df_train_labels_raw = pd.read_csv(
    METAPATH + "/train-annotations-human-imagelabels-boxable.csv")
print('OK!')


# validation files
print('Gathering all validation files...', end='')
df_val_bbox_raw = pd.read_csv(METAPATH + "/validation-annotations-bbox.csv")
df_val_labels_raw = pd.read_csv(
    METAPATH + "/validation-annotations-human-imagelabels-boxable.csv")
print('OK!')


# test files
print('Gathering all test files...', end='')
df_test_bbox_raw = pd.read_csv(METAPATH + "/test-annotations-bbox.csv")
df_test_labels_raw = pd.read_csv(
    METAPATH + "/test-annotations-human-imagelabels-boxable.csv")
print('OK!')

# downloaded images
if utils.all_images_downloaded():
    print("Gathering labels of all images downloaded...", end='')
    df_images_train = utils.images_downloaded('TRAIN')
    df_images_validation = utils.images_downloaded('VALIDATION')
    df_images_test = utils.images_downloaded('TEST')
    print('Ok!')
else:
    print("""
Unfortunately you did not download the files...
That's ok, you can follow, but some cells will not run properly.
    """)

### Assess

#### Explaining data

It is crucial to start the analysis to understand all the files downloaded.

When you run the command `make download-content`, you should notice that the files are divided into four blocks, METADATA, TRAIN, VALIDATION, and TEST.

The blocks TRAIN, VALIDATION, and TEST are about the images files, and there is nothing but that, the images.

**The block METADATA is a must-have content**. In it, we are going to find eight files.

**The first six files are essentially a split of two data**, the bounding box annotations *([type]-annotations-bbox.csv)* and the image-level annotations *([type]-annotations-human-imagelabels-boxable.csv)*.

Image-level Annotations are manual data labeled by a human (from google and a crowdsourced system). These labels try to represent what the image has. More about this dataset could be found in [here](https://storage.googleapis.com/openimages/web/factsfigures.html) in session **Image-level Labels**.

The Bounding Box Annotation is a dataset that defines, for each image, where are the objects annotated previously by the Image-level Annotation. These annotations are focused on the most specific labels in Image-level Annotations. More about this dataset could be founded [here](https://storage.googleapis.com/openimages/web/factsfigures.html) in session **Bonding Boxes**. 

These two datasets described above were divided by Google in three sets. Train, which is going to be used to train the model, Validation, which is going to be used to compare the models, and Test, which is going to be used to do the final evaluation of the chosen one model.

The final two files are about the classes themselves.

All the labels described in the paragraphs above are encoded representations of the classes. The CSV *class-descriptions-boxable.csv* maps each **encoded class name** (machine-understandable) **to a semantic class name** (human-understandable).

Lastly, the JSON file *bbox_labels_600_hierarchy.json* **describes a hierarchical tree of all classes**, describing a hierarchy of each class. This file shows us, for example, that Woman derives from Person.

#### Assessing Data

In [ ]:
#number of images by dataset
print(f"""Number of images: {(df_train_bbox_raw.shape[0]
                            + df_val_bbox_raw.shape[0]
                            + df_test_bbox_raw.shape[0]):,}""", end="\n"*2)

print(f"images in train: {df_train_bbox_raw.shape[0]:,}")
print(f"images in validation: {df_val_bbox_raw.shape[0]:,}")
print(f"images in test: {df_test_bbox_raw.shape[0]:,}")

#### Explaining more about the data

In [ ]:
# display classes and their encodes
print("####### CLASSES MAPPING #######")
print(f"total classes: {df_classes_raw.shape[0]}")
display(df_classes_raw.sample(2, random_state=17))

In the cell above, all datasets are described:

In [ ]:
# show info about all dfs
for k, df in {'Train Bounding Boxes': df_train_bbox_raw,
              'Train Labels': df_train_labels_raw,
              'Validation Bounding Boxes': df_val_bbox_raw,
              'Validation Labels': df_val_labels_raw,
              'Test Bounding Boxes': df_test_bbox_raw,
              'Test Labels': df_test_labels_raw}.items():
    print(f"####### {k.upper()} #######", end="\n"*2)
    print(f"shape: {df.shape[0]:,} rows, {df.shape[1]} columns")
    print(f"duplicated values: {df[df.duplicated(keep='first')].shape[0]} records",
          end="\n"*2)

    print("Unique Values:")
    for col in df.columns:
        print(
            f"   {str(col)+' ':-<15} {str(df[col].dtype).upper()+' ':-<10} Nulls = {df[col].isna().sum():,} | Uniques = {df[col].nunique():,}")
    display(df.sample(2, random_state=37))
    print('_'*80, end="\n"*2)

In [ ]:
# classes hierarchy
print("####### CLASSES Hierarchy #######", end="\n"*2)

num_classes, classes = utils.count_recursive(dict_hierarchy_raw)

print(f"There are {num_classes} classes in the JSON hierarchy",
      end="\n"*2)

print("The first node class encode is:",
      dict_hierarchy_raw['LabelName'], end="\n"*2)

# defining a node to consult
i=17

print(f"the {i}th son encode of the first node:",
      dict_hierarchy_raw['Subcategory'][i]['LabelName'])
print(f"The sons of the {i}th node:")
for subcat in dict_hierarchy_raw['Subcategory'][i]['Subcategory']:
    print(f"   {subcat}")

In [ ]:
# finding with some class in hierarchial has no human representation
for encoded_name in classes:
    try:
        semantic_name = utils.semantic_name(encoded_name)
    except KeyError:
        print(f"{encoded_name} - 'Undefined Human Representation'")

Import to notice that only the first node ("the father of all") has no semantic representation!

**Some classes could be in more then on subtree**, as shown above

In [ ]:
# duplicated paths
ambiguous_paths = [k for k, c in Counter(classes).items() if c > 1]

random.seed(57)
for cat in random.choices(ambiguous_paths, k=3):
    print(f"{utils.semantic_name(cat)}")
    paths = utils.node_path(dict_hierarchy_raw, cat)
    for path in paths:
        print(f"    " + " -> ".join([utils.semantic_name(k) for k in path]))

### Clean

## EDA